In [1]:
import dask.dataframe as dd
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import folium
from datetime import datetime
import time
import seaborn as sns
import os
import math
import xgboost as xgb
import matplotlib
matplotlib.use('nbagg')
import sklearn
import warnings
warnings.simplefilter('ignore')


In [2]:
import pandas as pd

# Load the April 2014 data
apr_2014 = pd.read_csv("uber-raw-data-apr14.csv")
apr_2014.columns = [c.replace(" ", "") for c in apr_2014.columns]

# Load the September 2014 data
may_2014 = pd.read_csv("uber-raw-data-sep14 (1).csv")
may_2014.columns = [c.replace(" ", "") for c in may_2014.columns]


In [3]:
 data_2014=pd.concat([apr_2014, may_2014], ignore_index=True)

In [4]:
data_2014



,Date/Time,Lat,Lon,Base
0,4/1/2014 0:11:00,40.7690,-73.9549,B02512
1,4/1/2014 0:17:00,40.7267,-74.0345,B02512
2,4/1/2014 0:21:00,40.7316,-73.9873,B02512
3,4/1/2014 0:28:00,40.7588,-73.9776,B02512
4,4/1/2014 0:33:00,40.7594,-73.9722,B02512
...,...,...,...,...
1592647,9/30/2014 22:57:00,40.7668,-73.9845,B02764
1592648,9/30/2014 22:57:00,40.6911,-74.1773,B02764
1592649,9/30/2014 22:58:00,40.8519,-73.9319,B02764
1592650,9/30/2014 22:58:00,40.7081,-74.0066,B02764


In [5]:
apr_2014 = data_2014[(((data_2014.Lat >= 40.5774) & (data_2014.Lat <= 40.9176)) & ((data_2014.Lon >= -74.15) & (data_2014.Lon <= -73.7004)))]
apr_2014

,Date/Time,Lat,Lon,Base
0,4/1/2014 0:11:00,40.7690,-73.9549,B02512
1,4/1/2014 0:17:00,40.7267,-74.0345,B02512
2,4/1/2014 0:21:00,40.7316,-73.9873,B02512
3,4/1/2014 0:28:00,40.7588,-73.9776,B02512
4,4/1/2014 0:33:00,40.7594,-73.9722,B02512
...,...,...,...,...
1592646,9/30/2014 22:57:00,40.7300,-73.9565,B02764
1592647,9/30/2014 22:57:00,40.7668,-73.9845,B02764
1592649,9/30/2014 22:58:00,40.8519,-73.9319,B02764
1592650,9/30/2014 22:58:00,40.7081,-74.0066,B02764


In [6]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import SGDRegressor
from sklearn.cluster import MiniBatchKMeans, KMeans


In [7]:
coord = apr_2014[["Lat", "Lon"]].values
neighbors = []

def min_distance(regionCenters, totalClusters):
    good_points = 0
    bad_points = 0
    less_dist = []
    more_dist = []
    min_distance = 100000  #any big number can be given here
    for i in range(totalClusters):
        good_points = 0
        bad_points = 0
        for j in range(totalClusters):
            if j != i:
                distance = gpxpy.geo.haversine_distance(latitude_1 = regionCenters[i][0], longitude_1 = regionCenters[i][1], latitude_2 = regionCenters[j][0], longitude_2 = regionCenters[j][1])
                #you can check the documentation of above "gpxpy.geo.haversine_distance" at "https://github.com/tkrajina/gpxpy/blob/master/gpxpy/geo.py"
                #"gpxpy.geo.haversine_distance" gives distance between two latitudes and longitudes in meters. So, we have to convert it into miles.
                distance = distance/(1.60934*1000)   #distance from meters to miles
                min_distance = min(min_distance, distance) #it will return minimum of "min_distance, distance".
                if distance < 2:
                    good_points += 1
                else:
                    bad_points += 1
        less_dist.append(good_points)
        more_dist.append(bad_points)
    print("On choosing a cluster size of {}".format(totalClusters))
    print("Avg. Number clusters within vicinity where inter cluster distance < 2 miles is {}".format(np.ceil(sum(less_dist)/len(less_dist))))
    print("Avg. Number clusters outside of vicinity where inter cluster distance > 2 miles is {}".format(np.ceil(sum(more_dist)/len(more_dist))))
    print("Minimum distance between any two clusters = {}".format(min_distance))
    print("-"*10)

def makingRegions(noOfRegions):
    regions = MiniBatchKMeans(n_clusters = noOfRegions, batch_size = 10000).fit(coord)
    regionCenters = regions.cluster_centers_
    totalClusters = len(regionCenters)
    return regionCenters, totalClusters

In [8]:
coord = apr_2014[["Lat", "Lon"]].values
regions = MiniBatchKMeans(n_clusters = 50, batch_size = 10000, random_state=0).fit(coord)
apr_2014["region_cluster"] = regions.predict(apr_2014[["Lat", "Lon"]])
apr_2014["cl_centers"]=regions.predict(apr_2014[["Lat", "Lon"]])

In [9]:
apr_2014.drop(columns=["cl_centers"], inplace=True)


In [10]:
apr_2014.head()


,Date/Time,Lat,Lon,Base,region_cluster
0,4/1/2014 0:11:00,40.7690,-73.9549,B02512,11
1,4/1/2014 0:17:00,40.7267,-74.0345,B02512,30
2,4/1/2014 0:21:00,40.7316,-73.9873,B02512,22
3,4/1/2014 0:28:00,40.7588,-73.9776,B02512,38
4,4/1/2014 0:33:00,40.7594,-73.9722,B02512,8


In [11]:
apr_2014["Date"]=pd.to_datetime(apr_2014["Date/Time"]).dt.strftime('%Y-%m-%d-%H')
apr_2014

,Date/Time,Lat,Lon,Base,region_cluster,Date
0,4/1/2014 0:11:00,40.7690,-73.9549,B02512,11,2014-04-01-00
1,4/1/2014 0:17:00,40.7267,-74.0345,B02512,30,2014-04-01-00
2,4/1/2014 0:21:00,40.7316,-73.9873,B02512,22,2014-04-01-00
3,4/1/2014 0:28:00,40.7588,-73.9776,B02512,38,2014-04-01-00
4,4/1/2014 0:33:00,40.7594,-73.9722,B02512,8,2014-04-01-00
...,...,...,...,...,...,...
1592646,9/30/2014 22:57:00,40.7300,-73.9565,B02764,29,2014-09-30-22
1592647,9/30/2014 22:57:00,40.7668,-73.9845,B02764,1,2014-09-30-22
1592649,9/30/2014 22:58:00,40.8519,-73.9319,B02764,21,2014-09-30-22
1592650,9/30/2014 22:58:00,40.7081,-74.0066,B02764,10,2014-09-30-22


In [12]:
count=apr_2014.groupby(["Date", "region_cluster"])["region_cluster"].count()
# count=data_2014.groupby('Date')['region_cluster'].count()
count

Date           region_cluster
2014-04-01-00  3                  6
               4                  5
               5                  2
               6                  1
               7                  1
                                 ..
2014-09-30-22  45                12
               46                 2
               47                13
               48                60
               49                16
Name: region_cluster, Length: 67275, dtype: int64

In [13]:
 data_2014=pd.concat([apr_2014, may_2014], ignore_index=True)

In [14]:
data_2014



,Date/Time,Lat,Lon,Base,region_cluster,Date
0,4/1/2014 0:11:00,40.7690,-73.9549,B02512,11.0,2014-04-01-00
1,4/1/2014 0:17:00,40.7267,-74.0345,B02512,30.0,2014-04-01-00
2,4/1/2014 0:21:00,40.7316,-73.9873,B02512,22.0,2014-04-01-00
3,4/1/2014 0:28:00,40.7588,-73.9776,B02512,38.0,2014-04-01-00
4,4/1/2014 0:33:00,40.7594,-73.9722,B02512,8.0,2014-04-01-00
...,...,...,...,...,...,...
2597436,9/30/2014 22:57:00,40.7668,-73.9845,B02764,NaN,NaN
2597437,9/30/2014 22:57:00,40.6911,-74.1773,B02764,NaN,NaN
2597438,9/30/2014 22:58:00,40.8519,-73.9319,B02764,NaN,NaN
2597439,9/30/2014 22:58:00,40.7081,-74.0066,B02764,NaN,NaN


In [15]:
apr_2014 = data_2014[(((data_2014.Lat >= 40.5774) & (data_2014.Lat <= 40.9176)) & ((data_2014.Lon >= -74.15) & (data_2014.Lon <= -73.7004)))]
apr_2014

,Date/Time,Lat,Lon,Base,region_cluster,Date
0,4/1/2014 0:11:00,40.7690,-73.9549,B02512,11.0,2014-04-01-00
1,4/1/2014 0:17:00,40.7267,-74.0345,B02512,30.0,2014-04-01-00
2,4/1/2014 0:21:00,40.7316,-73.9873,B02512,22.0,2014-04-01-00
3,4/1/2014 0:28:00,40.7588,-73.9776,B02512,38.0,2014-04-01-00
4,4/1/2014 0:33:00,40.7594,-73.9722,B02512,8.0,2014-04-01-00
...,...,...,...,...,...,...
2597435,9/30/2014 22:57:00,40.7300,-73.9565,B02764,NaN,NaN
2597436,9/30/2014 22:57:00,40.7668,-73.9845,B02764,NaN,NaN
2597438,9/30/2014 22:58:00,40.8519,-73.9319,B02764,NaN,NaN
2597439,9/30/2014 22:58:00,40.7081,-74.0066,B02764,NaN,NaN


In [16]:
 data_2014=pd.concat([apr_2014, may_2014], ignore_index=True)

In [17]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import SGDRegressor
from sklearn.cluster import MiniBatchKMeans, KMeans

In [18]:
coord = apr_2014[["Lat", "Lon"]].values
neighbors = []

def min_distance(regionCenters, totalClusters):
    good_points = 0
    bad_points = 0
    less_dist = []
    more_dist = []
    min_distance = 100000  #any big number can be given here
    for i in range(totalClusters):
        good_points = 0
        bad_points = 0
        for j in range(totalClusters):
            if j != i:
                distance = gpxpy.geo.haversine_distance(latitude_1 = regionCenters[i][0], longitude_1 = regionCenters[i][1], latitude_2 = regionCenters[j][0], longitude_2 = regionCenters[j][1])
                #you can check the documentation of above "gpxpy.geo.haversine_distance" at "https://github.com/tkrajina/gpxpy/blob/master/gpxpy/geo.py"
                #"gpxpy.geo.haversine_distance" gives distance between two latitudes and longitudes in meters. So, we have to convert it into miles.
                distance = distance/(1.60934*1000)   #distance from meters to miles
                min_distance = min(min_distance, distance) #it will return minimum of "min_distance, distance".
                if distance < 2:
                    good_points += 1
                else:
                    bad_points += 1
        less_dist.append(good_points)
        more_dist.append(bad_points)
    print("On choosing a cluster size of {}".format(totalClusters))
    print("Avg. Number clusters within vicinity where inter cluster distance < 2 miles is {}".format(np.ceil(sum(less_dist)/len(less_dist))))
    print("Avg. Number clusters outside of vicinity where inter cluster distance > 2 miles is {}".format(np.ceil(sum(more_dist)/len(more_dist))))
    print("Minimum distance between any two clusters = {}".format(min_distance))
    print("-"*10)

def makingRegions(noOfRegions):
    regions = MiniBatchKMeans(n_clusters = noOfRegions, batch_size = 10000).fit(coord)
    regionCenters = regions.cluster_centers_
    totalClusters = len(regionCenters)
    return regionCenters, totalClusters

In [19]:
coord = apr_2014[["Lat", "Lon"]].values
regions = MiniBatchKMeans(n_clusters = 50, batch_size = 10000, random_state=0).fit(coord)
apr_2014["region_cluster"] = regions.predict(apr_2014[["Lat", "Lon"]])
apr_2014["cl_centers"]=regions.predict(apr_2014[["Lat", "Lon"]])

In [20]:
apr_2014.drop(columns=["cl_centers"], inplace=True)


In [21]:
apr_2014.head()

,Date/Time,Lat,Lon,Base,region_cluster,Date
0,4/1/2014 0:11:00,40.7690,-73.9549,B02512,13,2014-04-01-00
1,4/1/2014 0:17:00,40.7267,-74.0345,B02512,21,2014-04-01-00
2,4/1/2014 0:21:00,40.7316,-73.9873,B02512,16,2014-04-01-00
3,4/1/2014 0:28:00,40.7588,-73.9776,B02512,44,2014-04-01-00
4,4/1/2014 0:33:00,40.7594,-73.9722,B02512,44,2014-04-01-00


In [22]:
apr_2014["Date"]=pd.to_datetime(apr_2014["Date/Time"]).dt.strftime('%Y-%m-%d-%H')
apr_2014

,Date/Time,Lat,Lon,Base,region_cluster,Date
0,4/1/2014 0:11:00,40.7690,-73.9549,B02512,13,2014-04-01-00
1,4/1/2014 0:17:00,40.7267,-74.0345,B02512,21,2014-04-01-00
2,4/1/2014 0:21:00,40.7316,-73.9873,B02512,16,2014-04-01-00
3,4/1/2014 0:28:00,40.7588,-73.9776,B02512,44,2014-04-01-00
4,4/1/2014 0:33:00,40.7594,-73.9722,B02512,44,2014-04-01-00
...,...,...,...,...,...,...
2597435,9/30/2014 22:57:00,40.7300,-73.9565,B02764,46,2014-09-30-22
2597436,9/30/2014 22:57:00,40.7668,-73.9845,B02764,24,2014-09-30-22
2597438,9/30/2014 22:58:00,40.8519,-73.9319,B02764,6,2014-09-30-22
2597439,9/30/2014 22:58:00,40.7081,-74.0066,B02764,17,2014-09-30-22


In [23]:
count=apr_2014.groupby(["Date", "region_cluster"])["region_cluster"].count()
# count=data_2014.groupby('Date')['region_cluster'].count()
count

Date           region_cluster
2014-04-01-00  0                   5
               1                   5
               3                   3
               5                   2
               7                   1
                                ... 
2014-09-30-22  45                 96
               46                 20
               47                 20
               48                104
               49                 22
Name: region_cluster, Length: 66975, dtype: int64

In [24]:
data_2014_new=apr_2014.groupby(['Date','region_cluster']).size().reset_index(name='count')
data_2014_new

,Date,region_cluster,count
0,2014-04-01-00,0,5
1,2014-04-01-00,1,5
2,2014-04-01-00,3,3
3,2014-04-01-00,5,2
4,2014-04-01-00,7,1
...,...,...,...
66970,2014-09-30-22,45,96
66971,2014-09-30-22,46,20
66972,2014-09-30-22,47,20
66973,2014-09-30-22,48,104


In [25]:
date=pd.to_datetime(data_2014_new['Date'], format='%Y-%m-%d-%H')
data_2014_new['Month']=date.dt.month
data_2014_new['Day']=date.dt.day
data_2014_new['Hour']=date.dt.hour
temp=pd.DatetimeIndex(date)
data_2014_new['Weekday']=temp.weekday
data_2014_new

,Date,region_cluster,count,Month,Day,Hour,Weekday
0,2014-04-01-00,0,5,4,1,0,1
1,2014-04-01-00,1,5,4,1,0,1
2,2014-04-01-00,3,3,4,1,0,1
3,2014-04-01-00,5,2,4,1,0,1
4,2014-04-01-00,7,1,4,1,0,1
...,...,...,...,...,...,...,...
66970,2014-09-30-22,45,96,9,30,22,1
66971,2014-09-30-22,46,20,9,30,22,1
66972,2014-09-30-22,47,20,9,30,22,1
66973,2014-09-30-22,48,104,9,30,22,1


In [26]:
data_model=data_2014_new[['Month','Weekday', 'Hour', 'Day','region_cluster','count']]
data_model.tail()

,Month,Weekday,Hour,Day,region_cluster,count
66970,9,1,22,30,45,96
66971,9,1,22,30,46,20
66972,9,1,22,30,47,20
66973,9,1,22,30,48,104
66974,9,1,22,30,49,22


In [27]:
target = data_model["count"]


In [28]:
add_dummies = pd.get_dummies(data_model['region_cluster'])
data_model = pd.concat([data_model, add_dummies], axis=1)
data_model.drop(['region_cluster'], inplace=True, axis=1)
target=data_model[['count']]
data=data_model[[col for col in data_model.columns if col not in ['count']]]

In [29]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(data_model, target, test_size=0.2, random_state=0)
from scipy import stats
# from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

In [30]:
x_train.columns

Index([  'Month', 'Weekday',    'Hour',     'Day',   'count',         0,
               1,         2,         3,         4,         5,         6,
               7,         8,         9,        10,        11,        12,
              13,        14,        15,        16,        17,        18,
              19,        20,        21,        22,        23,        24,
              25,        26,        27,        28,        29,        30,
              31,        32,        33,        34,        35,        36,
              37,        38,        39,        40,        41,        42,
              43,        44,        45,        46,        47,        48,
              49],
      dtype='object')

In [31]:
 print("x_train shape:", x_train.shape)


x_train shape: (53580, 55)


In [32]:
#BAGGING WITH EXTRA TREE CLASSIFIERS.

import pandas as pd
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# Initialize KNN Regressor
knn_regressor = KNeighborsRegressor(n_neighbors=5)  # You can adjust the number of neighbors as needed

# Convert column names to strings
x_train.columns = x_train.columns.astype(str)
x_test.columns = x_test.columns.astype(str)

# Train the KNN Regressor
knn_regressor.fit(x_train, y_train)

# Define threshold for classifying predictions
threshold = 0.5  # Adjust threshold as needed

# Convert continuous predictions to discrete classes
y_pred_train_class = (knn_regressor.predict(x_train) > threshold).astype(int)
y_pred_test_class = (knn_regressor.predict(x_test) > threshold).astype(int)

# Calculate evaluation metrics
accuracy = accuracy_score(y_test, y_pred_test_class)
precision = precision_score(y_test, y_pred_test_class, average='macro')
recall = recall_score(y_test, y_pred_test_class, average='macro')
f1 = f1_score(y_test, y_pred_test_class, average='macro')


# Create a DataFrame for precision and recall values
metrics_df = pd.DataFrame({
    'Metric': ['Accuracy', 'Precision', 'Recall', 'F1 Score'],
    'Overall': [accuracy, precision, recall, f1]
})

print("Overall Evaluation Metrics:")
print(metrics_df)

# Calculate confusion matrix
cm = confusion_matrix(y_test, y_pred_test_class)

# Print confusion matrix
print("\nConfusion Matrix:")
print(cm)

# Plot confusion matrix
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", cbar=False)
plt.xlabel('Predicted labels')
plt.ylabel('True labels')
plt.title('Confusion Matrix')
plt.show()


Overall Evaluation Metrics:
      Metric   Overall
0   Accuracy  0.059127
1  Precision  0.000203
2     Recall  0.003436
3   F1 Score  0.000384

Confusion Matrix:
[[792   0   0 ...   0   0   0]
 [942   0   0 ...   0   0   0]
 [537   0   0 ...   0   0   0]
 ...
 [  1   0   0 ...   0   0   0]
 [  1   0   0 ...   0   0   0]
 [  1   0   0 ...   0   0   0]]


<IPython.core.display.Javascript object>

In [33]:
from sklearn.ensemble import ExtraTreesClassifier, BaggingClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_score, recall_score, f1_score

# Define the base classifier
base_classifier = ExtraTreesClassifier(n_estimators=10, random_state=42)  # You can adjust parameters as needed

# Define the Bagging Classifier
bagging_classifier = BaggingClassifier(base_estimator=base_classifier, n_estimators=10, random_state=42)

# Pipeline for preprocessing with Bagging Classifier
pipeline = Pipeline([
    ('scaler', StandardScaler()),  # Standardize features by removing the mean and scaling to unit variance
    ('bagging_classifier', bagging_classifier)
])

# Train the model
pipeline.fit(x_train, y_train)

# Predict on the test set
y_pred = pipeline.predict(x_test)

# Calculate precision, recall, and F1 score
precision = precision_score(y_test, y_pred, average='macro')
recall = recall_score(y_test, y_pred, average='macro')
f1 = f1_score(y_test, y_pred, average='macro')

print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)


Precision: 0.1626174414608794
Recall: 0.16824336765284567
F1 Score: 0.16231912263290257


In [34]:
# Feature selection using PCA Here we had taken PCA 5 Components
from sklearn.decomposition import PCA


x_train.columns = x_train.columns.astype(str)
x_test.columns = x_test.columns.astype(str)
pca = PCA(n_components=5)
x_train_pca = pca.fit_transform(x_train)
x_test_pca = pca.transform(x_test)

In [35]:

from sklearn.decomposition import PCA
from sklearn.metrics import r2_score
import xgboost as xgb
XGB = xgb.XGBRegressor(colsample_bytree=0.7, learning_rate=0.07, max_depth=8, min_child_weight=4, n_estimators=500, nthread=4, silent=True, subsample=0.7)
XGB.fit(x_train_pca, y_train)
y_pred_test_original = XGB.predict(x_test_pca)
r2_test_original = r2_score(y_test, y_pred_test_original)
print(" R-squared score :", r2_test_original)


 R-squared score : 0.9989697587913496


In [36]:
from sklearn.metrics import precision_score, recall_score, f1_score

# Define thresholds to convert continuous predictions to discrete classes
threshold_low = 0
threshold_high = 1  # You may adjust the thresholds based on your specific problem

# Convert continuous predictions to discrete classes based on the thresholds
y_pred_test_class = ((y_pred_test_original >= threshold_high) * 1).astype(int)

# Calculate precision, recall, and F1 score
precision = precision_score(y_test, y_pred_test_class, average='micro')  # Assuming binary classification
recall = recall_score(y_test, y_pred_test_class, average='micro')
f1 = f1_score(y_test, y_pred_test_class, average='micro')

print("Precision Score:", precision)
print("Recall Score:", recall)
print("F1 Score:", f1)


Precision Score: 0.027696901829040686
Recall Score: 0.027696901829040686
F1 Score: 0.027696901829040686


In [37]:
#GRADIENT BOOSTING
from sklearn.decomposition import PCA
from sklearn.metrics import r2_score
from sklearn.ensemble import GradientBoostingRegressor
GBR = GradientBoostingRegressor(n_estimators=500, learning_rate=0.07, max_depth=8, min_samples_split=4, min_samples_leaf=1, subsample=0.7, random_state=42)
GBR.fit(x_train_pca, y_train)
y_pred_test_original = GBR.predict(x_test_pca)
r2_test_original = r2_score(y_test, y_pred_test_original)
print(" R-squared score :", r2_test_original)

 R-squared score : 0.9999988111083047


In [38]:
from sklearn.metrics import precision_score, recall_score, f1_score

# Define thresholds to convert continuous predictions to discrete classes
threshold_low = 0
threshold_high = 1  # You may adjust the thresholds based on your specific problem

# Convert continuous predictions to discrete classes based on the thresholds
y_pred_class = ((y_pred_test_original >= threshold_high) * 1).astype(int)

# Calculate precision, recall, and F1 score
precision = precision_score(y_test, y_pred_class, average='micro')  # Assuming binary classification
recall = recall_score(y_test, y_pred_class, average='micro')
f1 = f1_score(y_test, y_pred_class, average='micro')

print("Precision Score:", precision)
print("Recall Score:", recall)
print("F1 Score:", f1)


Precision Score: 0.05912653975363942
Recall Score: 0.05912653975363942
F1 Score: 0.05912653975363942


In [39]:
#ADA BOOSTING
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score
base_classifier = DecisionTreeClassifier(max_depth=10)
adaboost_classifier = AdaBoostClassifier(base_estimator=base_classifier, n_estimators=25, random_state=42)
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('pca', pca),
    ('adaboost_classifier', adaboost_classifier)
])
pipeline.fit(x_train_pca, y_train)
y_pred = pipeline.predict(x_test_pca)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.9052631578947369


In [40]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_score, recall_score, f1_score
base_classifier = DecisionTreeClassifier(max_depth=3)
adaboost_classifier = AdaBoostClassifier(base_estimator=base_classifier, n_estimators=50, random_state=42)
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('pca', pca),
    ('adaboost_classifier', adaboost_classifier)
])
pipeline.fit(x_train_pca, y_train)
y_pred = pipeline.predict(x_test_pca)
precision = precision_score(y_test, y_pred, average='macro')
recall = recall_score(y_test, y_pred, average='macro')
f1 = f1_score(y_test, y_pred, average='macro')
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)


Precision: 0.13546850371387797
Recall: 0.16943521594684385
F1 Score: 0.14293321972355283


In [41]:
#BAGGING
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
base_classifier = DecisionTreeClassifier(max_depth=10)
bagging_classifier = BaggingClassifier(base_estimator=base_classifier, n_estimators=40, random_state=42)
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('pca', pca),
    ('bagging_classifier', bagging_classifier)
])
pipeline.fit(x_train_pca, y_train)
y_pred = pipeline.predict(x_test_pca)
accuracy = accuracy_score(y_test, y_pred)
print(" Bagging Accuracy:", accuracy)

 Bagging Accuracy: 0.5593131765584173


In [42]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_score, recall_score, f1_score
base_classifier = DecisionTreeClassifier(max_depth=3)
bagging_classifier = BaggingClassifier(base_estimator=base_classifier, n_estimators=50, random_state=42)
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('pca', pca),
    ('bagging_classifier', bagging_classifier)
])
pipeline.fit(x_train_pca, y_train)
y_pred = pipeline.predict(x_test_pca)
precision = precision_score(y_test, y_pred, average='macro')
recall = recall_score(y_test, y_pred, average='macro')
f1 = f1_score(y_test, y_pred, average='macro')
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

Precision: 0.016177747921982472
Recall: 0.020618556701030927
F1 Score: 0.016795259028237947


In [43]:
#Taking PCA with 4 Components
from sklearn.decomposition import PCA
x_train.columns = x_train.columns.astype(str)
x_test.columns = x_test.columns.astype(str)
pca = PCA(n_components=4)
x_train_pca = pca.fit_transform(x_train)
x_test_pca = pca.transform(x_test)

In [44]:
#XG BOOSING
from sklearn.decomposition import PCA
from sklearn.metrics import r2_score
import xgboost as xgb
XGB = xgb.XGBRegressor(colsample_bytree=0.7, learning_rate=0.07, max_depth=8, min_child_weight=4, n_estimators=500, nthread=4, silent=True, subsample=0.7)
XGB.fit(x_train_pca, y_train)
y_pred_test_original = XGB.predict(x_test_pca)
r2_test_original = r2_score(y_test, y_pred_test_original)
print(" R-squared score :", r2_test_original)


 R-squared score : 0.9983782724326895


In [45]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
mse = mean_squared_error(y_test, y_pred_test_original)
print("Mean Squared Error (MSE):", mse)
mae = mean_absolute_error(y_test, y_pred_test_original)
print("Mean Absolute Error (MAE):", mae)

Mean Squared Error (MSE): 5.373657239698833
Mean Absolute Error (MAE): 0.7294445097730115


In [46]:
#GRADIENT BOOSTING
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.ensemble import GradientBoostingRegressor
GBR = GradientBoostingRegressor(n_estimators=500, learning_rate=0.07, max_depth=8, min_samples_split=4, min_samples_leaf=1, subsample=0.7, random_state=42)
GBR.fit(x_train_pca, y_train)
y_pred_test_original = GBR.predict(x_test_pca)
mse = mean_squared_error(y_test, y_pred_test_original)
print("Mean Squared Error (MSE):", mse)
mae = mean_absolute_error(y_test, y_pred_test_original)
print("Mean Absolute Error (MAE):", mae)

Mean Squared Error (MSE): 0.005270172025004027
Mean Absolute Error (MAE): 0.002792584680111541


In [47]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.ensemble import GradientBoostingRegressor
GBR = GradientBoostingRegressor(n_estimators=500, learning_rate=0.07, max_depth=8, min_samples_split=4, min_samples_leaf=1, subsample=0.7, random_state=42)
GBR.fit(x_train_pca, y_train)
y_pred_test_original = GBR.predict(x_test_pca)
mse = mean_squared_error(y_test, y_pred_test_original)
print("Mean Squared Error (MSE):", mse)
mae = mean_absolute_error(y_test, y_pred_test_original)
print("Mean Absolute Error (MAE):", mae)

Mean Squared Error (MSE): 0.005270172025004027
Mean Absolute Error (MAE): 0.002792584680111541


In [48]:
# KNN
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score
base_classifier = KNeighborsClassifier(n_neighbors=5)
bagging_classifier = BaggingClassifier(base_estimator=base_classifier, n_estimators=50, random_state=42)
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('pca', pca),
    ('bagging_classifier', bagging_classifier)
])
pipeline.fit(x_train_pca, y_train)
y_pred = pipeline.predict(x_test_pca)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.36998880179171334


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

from sklearn.decomposition import PCA

# Initialize lists to store training and testing accuracies
train_accuracies = []
test_accuracies = []

# Define a range of number of estimators
estimator_range = range(1, 51)

# Iterate over each number of estimators
for n_estimators in estimator_range:
    # Define Bagging Classifier with current number of estimators
    bagging_classifier = BaggingClassifier(base_estimator=base_classifier, n_estimators=n_estimators, random_state=42)
    pca = PCA(n_components=4)
    # Define the pipeline
    pipeline = Pipeline([
        ('scaler', StandardScaler()),
        ('pca', pca),
        ('bagging_classifier', bagging_classifier)
    ])

    # Train the model
    pipeline.fit(x_train_pca, y_train)

    # Predict on the training and testing data
    y_train_pred = pipeline.predict(x_train_pca)
    y_test_pred = pipeline.predict(x_test_pca)

    # Calculate accuracy scores
    train_accuracy = accuracy_score(y_train, y_train_pred)
    test_accuracy = accuracy_score(y_test, y_test_pred)

    # Append accuracies to the lists
    train_accuracies.append(train_accuracy)
    test_accuracies.append(test_accuracy)

# Plot the training and testing accuracies against number of estimators
plt.figure(figsize=(10, 6))
plt.plot(estimator_range, train_accuracies, label='Training Accuracy')
plt.plot(estimator_range, test_accuracies, label='Testing Accuracy')
plt.xlabel('Number of Estimators')
plt.ylabel('Accuracy')
plt.title('Training and Testing Accuracies vs. Number of Estimators')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:

from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_score, recall_score, f1_score
base_classifier = KNeighborsClassifier(n_neighbors=5)
bagging_classifier = BaggingClassifier(base_estimator=base_classifier, n_estimators=50, random_state=42)
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('pca', pca),
    ('bagging_classifier', bagging_classifier)
])
pipeline.fit(x_train_pca, y_train)
y_pred = pipeline.predict(x_test_pca)
precision = precision_score(y_test, y_pred, average='macro')
recall = recall_score(y_test, y_pred, average='macro')
f1 = f1_score(y_test, y_pred, average='macro')
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

In [ ]:
# ADA BOOST
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score
base_classifier = DecisionTreeClassifier(max_depth=3)
adaboost_classifier = AdaBoostClassifier(base_estimator=base_classifier, n_estimators=50, random_state=42)
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('pca', pca),
    ('adaboost_classifier', adaboost_classifier)
])
pipeline.fit(x_train_pca, y_train)
y_pred = pipeline.predict(x_test_pca)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_score, recall_score, f1_score
base_classifier = DecisionTreeClassifier(max_depth=3)
adaboost_classifier = AdaBoostClassifier(base_estimator=base_classifier, n_estimators=50, random_state=42)
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('pca', pca),
    ('adaboost_classifier', adaboost_classifier)
])
pipeline.fit(x_train_pca, y_train)
y_pred = pipeline.predict(x_test_pca)
precision = precision_score(y_test, y_pred, average='macro')
recall = recall_score(y_test, y_pred, average='macro')
f1 = f1_score(y_test, y_pred, average='macro')
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)



In [ ]:
# BAGGING
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
base_classifier = DecisionTreeClassifier(max_depth=3)
bagging_classifier = BaggingClassifier(base_estimator=base_classifier, n_estimators=50, random_state=42)
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('pca', pca),
    ('bagging_classifier', bagging_classifier)
])
pipeline.fit(x_train_pca, y_train)
y_pred = pipeline.predict(x_test_pca)
accuracy = accuracy_score(y_test, y_pred)
print(" Bagging Accuracy:", accuracy)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_score, recall_score, f1_score
base_classifier = DecisionTreeClassifier(max_depth=3)
bagging_classifier = BaggingClassifier(base_estimator=base_classifier, n_estimators=50, random_state=42)
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('pca', pca),
    ('bagging_classifier', bagging_classifier)
])
pipeline.fit(x_train_pca, y_train)
y_pred = pipeline.predict(x_test_pca)
precision = precision_score(y_test, y_pred, average='macro')
recall = recall_score(y_test, y_pred, average='macro')
f1 = f1_score(y_test, y_pred, average='macro')
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

In [ ]:
# PCA WITH 6 COMPONENTS

x_train.columns = x_train.columns.astype(str)
x_test.columns = x_test.columns.astype(str)
pca = PCA(n_components=6)
x_train_pca = pca.fit_transform(x_train)
x_test_pca = pca.transform(x_test)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

from sklearn.decomposition import PCA

# Initialize lists to store training and testing accuracies
train_accuracies = []
test_accuracies = []

# Define a range of number of estimators
estimator_range = range(1, 51)

# Iterate over each number of estimators
for n_estimators in estimator_range:
    # Define Bagging Classifier with current number of estimators
    bagging_classifier = BaggingClassifier(base_estimator=base_classifier, n_estimators=n_estimators, random_state=42)
    pca = PCA(n_components=10)
    # Define the pipeline
    pipeline = Pipeline([
        ('scaler', StandardScaler()),
        ('pca', pca),
        ('bagging_classifier', bagging_classifier)
    ])

    # Train the model
    pipeline.fit(x_train_pca, y_train)

    # Predict on the training and testing data
    y_train_pred = pipeline.predict(x_train_pca)
    y_test_pred = pipeline.predict(x_test_pca)

    # Calculate accuracy scores
    train_accuracy = accuracy_score(y_train, y_train_pred)
    test_accuracy = accuracy_score(y_test, y_test_pred)

    # Append accuracies to the lists
    train_accuracies.append(train_accuracy)
    test_accuracies.append(test_accuracy)

# Plot the training and testing accuracies against number of estimators
plt.figure(figsize=(10, 6))
plt.plot(estimator_range, train_accuracies, label='Training Accuracy')
plt.plot(estimator_range, test_accuracies, label='Testing Accuracy')
plt.xlabel('Number of Estimators')
plt.ylabel('Accuracy')
plt.title('Training and Testing Accuracies vs. Number of Estimators')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix

# Calculate confusion matrix
cm_nb = confusion_matrix(y_test, ypred)

# Plot confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm_nb, annot=True, fmt='d', cmap='Blues', xticklabels=['Not Grow', 'Grow'], yticklabels=['Not Grow', 'Grow'])
plt.xlabel('Predicted labels')
plt.ylabel('True labels')
plt.title('Confusion Matrix for Naive Bayes Model')
plt.show()

In [ ]:
# XG BOOST
from sklearn.decomposition import PCA
from sklearn.metrics import r2_score
import xgboost as xgb
XGB = xgb.XGBRegressor(colsample_bytree=0.7, learning_rate=0.07, max_depth=8, min_child_weight=4, n_estimators=500, nthread=4, silent=True, subsample=0.7)
XGB.fit(x_train_pca, y_train)
y_pred_test_original = XGB.predict(x_test_pca)
r2_test_original = r2_score(y_test, y_pred_test_original)
print(" R-squared score :", r2_test_original)

In [ ]:
# GRADIENT BOOSTING
from sklearn.decomposition import PCA
from sklearn.metrics import r2_score
from sklearn.ensemble import GradientBoostingRegressor
GBR = GradientBoostingRegressor(n_estimators=500, learning_rate=0.07, max_depth=8, min_samples_split=4, min_samples_leaf=1, subsample=0.7, random_state=42)
GBR.fit(x_train_pca, y_train)
y_pred_test_original = GBR.predict(x_test_pca)
r2_test_original = r2_score(y_test, y_pred_test_original)
print(" R-squared score :", r2_test_original)

In [ ]:
#KNN
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score
base_classifier = KNeighborsClassifier(n_neighbors=5)
bagging_classifier = BaggingClassifier(base_estimator=base_classifier, n_estimators=5, random_state=42)
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('pca', pca),
    ('bagging_classifier', bagging_classifier)
])
pipeline.fit(x_train_pca, y_train)
y_pred = pipeline.predict(x_test_pca)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


In [ ]:
# ADA BOOST
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score
base_classifier = DecisionTreeClassifier(max_depth=3)
adaboost_classifier = AdaBoostClassifier(base_estimator=base_classifier, n_estimators=50, random_state=42)
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('pca', pca),
    ('adaboost_classifier', adaboost_classifier)
])
pipeline.fit(x_train_pca, y_train)
y_pred = pipeline.predict(x_test_pca)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

In [ ]:
#BAGGING
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score
base_classifier = DecisionTreeClassifier(max_depth=3)
adaboost_classifier = AdaBoostClassifier(base_estimator=base_classifier, n_estimators=50, random_state=42)
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('pca', pca),
    ('adaboost_classifier', adaboost_classifier)
])
pipeline.fit(x_train_pca, y_train)
y_pred = pipeline.predict(x_test_pca)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

In [ ]:
# TAKING PCA WITH FOUR COMPONENTS
from sklearn.decomposition import PCA

x_train.columns = x_train.columns.astype(str)
x_test.columns = x_test.columns.astype(str)
pca = PCA(n_components=3)
x_train_pca = pca.fit_transform(x_train)
x_test_pca = pca.transform(x_test)

In [ ]:
#XG BOOST
from sklearn.decomposition import PCA
from sklearn.metrics import r2_score
import xgboost as xgb
XGB = xgb.XGBRegressor(colsample_bytree=0.7, learning_rate=0.07, max_depth=8, min_child_weight=4, n_estimators=500, nthread=4, silent=True, subsample=0.7)
XGB.fit(x_train_pca, y_train)
y_pred_test_original = XGB.predict(x_test_pca)
r2_test_original = r2_score(y_test, y_pred_test_original)
print(" R-squared score :", r2_test_original)

In [ ]:
#GRADIENT BOOSTING
from sklearn.decomposition import PCA
from sklearn.metrics import r2_score
from sklearn.ensemble import GradientBoostingRegressor
GBR = GradientBoostingRegressor(n_estimators=500, learning_rate=0.07, max_depth=8, min_samples_split=4, min_samples_leaf=1, subsample=0.7, random_state=42)
GBR.fit(x_train_pca, y_train)
y_pred_test_original = GBR.predict(x_test_pca)
r2_test_original = r2_score(y_test, y_pred_test_original)
print(" R-squared score :", r2_test_original)

In [ ]:
#KNN 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score
base_classifier = KNeighborsClassifier(n_neighbors=5)
bagging_classifier = BaggingClassifier(base_estimator=base_classifier, n_estimators=50, random_state=42)
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('pca', pca),
    ('bagging_classifier', bagging_classifier)
])
pipeline.fit(x_train_pca, y_train)
y_pred = pipeline.predict(x_test_pca)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

In [ ]:
#ADA BOOST
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score
base_classifier = DecisionTreeClassifier(max_depth=3)
adaboost_classifier = AdaBoostClassifier(base_estimator=base_classifier, n_estimators=50, random_state=42)
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('pca', pca),
    ('adaboost_classifier', adaboost_classifier)
])
pipeline.fit(x_train_pca, y_train)
y_pred = pipeline.predict(x_test_pca)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

In [ ]:
#BAGGING 
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
base_classifier = DecisionTreeClassifier(max_depth=3)
bagging_classifier = BaggingClassifier(base_estimator=base_classifier, n_estimators=50, random_state=42)
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('pca', pca),
    ('bagging_classifier', bagging_classifier)
])
pipeline.fit(x_train_pca, y_train)
y_pred = pipeline.predict(x_test_pca)
accuracy = accuracy_score(y_test, y_pred)
print(" Bagging Accuracy:", accuracy)

In [ ]:
# TAKING PCA WITH TWO COMPONENTS 

from sklearn.decomposition import PCA
x_train.columns = x_train.columns.astype(str)
x_test.columns = x_test.columns.astype(str)
pca = PCA(n_components=2)
x_train_pca = pca.fit_transform(x_train)
x_test_pca = pca.transform(x_test)

In [ ]:
# XG BOOST

from sklearn.decomposition import PCA
from sklearn.metrics import r2_score
import xgboost as xgb
XGB = xgb.XGBRegressor(colsample_bytree=0.7, learning_rate=0.07, max_depth=8, min_child_weight=4, n_estimators=500, nthread=4, silent=True, subsample=0.7)
XGB.fit(x_train_pca, y_train)
y_pred_test_original = XGB.predict(x_test_pca)
r2_test_original = r2_score(y_test, y_pred_test_original)
print(" R-squared score :", r2_test_original)

In [ ]:
#GRADIENT BOOSTING
from sklearn.decomposition import PCA
from sklearn.metrics import r2_score
from sklearn.ensemble import GradientBoostingRegressor
GBR = GradientBoostingRegressor(n_estimators=500, learning_rate=0.07, max_depth=8, min_samples_split=4, min_samples_leaf=1, subsample=0.7, random_state=42)
GBR.fit(x_train_pca, y_train)
y_pred_test_original = GBR.predict(x_test_pca)
r2_test_original = r2_score(y_test, y_pred_test_original)
print(" R-squared score :", r2_test_original)


In [ ]:
#KNN 

from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score
base_classifier = KNeighborsClassifier(n_neighbors=5)
bagging_classifier = BaggingClassifier(base_estimator=base_classifier, n_estimators=50, random_state=42)
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('pca', pca),
    ('bagging_classifier', bagging_classifier)
])
pipeline.fit(x_train_pca, y_train)
y_pred = pipeline.predict(x_test_pca)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

In [ ]:
# ADA BOOST

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score
base_classifier = DecisionTreeClassifier(max_depth=3)
adaboost_classifier = AdaBoostClassifier(base_estimator=base_classifier, n_estimators=50, random_state=42)
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('pca', pca),
    ('adaboost_classifier', adaboost_classifier)
])
pipeline.fit(x_train_pca, y_train)
y_pred = pipeline.predict(x_test_pca)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


In [ ]:
#BAGGING 

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
base_classifier = DecisionTreeClassifier(max_depth=3)
bagging_classifier = BaggingClassifier(base_estimator=base_classifier, n_estimators=50, random_state=42)
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('pca', pca),
    ('bagging_classifier', bagging_classifier)
])
pipeline.fit(x_train_pca, y_train)
y_pred = pipeline.predict(x_test_pca)
accuracy = accuracy_score(y_test, y_pred)
print(" Bagging Accuracy:", accuracy)

In [ ]:
# TAKING PCA WITH ONE COMPONENT 

from sklearn.decomposition import PCA
x_train.columns = x_train.columns.astype(str)
x_test.columns = x_test.columns.astype(str)
pca = PCA(n_components=1)
x_train_pca = pca.fit_transform(x_train)
x_test_pca = pca.transform(x_test)

In [ ]:
# BAGGING

from sklearn.decomposition import PCA
from sklearn.metrics import r2_score
import xgboost as xgb
XGB = xgb.XGBRegressor(colsample_bytree=0.7, learning_rate=0.07, max_depth=8, min_child_weight=4, n_estimators=500, nthread=4, silent=True, subsample=0.7)
XGB.fit(x_train_pca, y_train)
y_pred_test_original = XGB.predict(x_test_pca)
r2_test_original = r2_score(y_test, y_pred_test_original)
print(" R-squared score :", r2_test_original)


In [ ]:
#ADA BOOSTING

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score
base_classifier = DecisionTreeClassifier(max_depth=3)
adaboost_classifier = AdaBoostClassifier(base_estimator=base_classifier, n_estimators=50, random_state=42)
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('pca', pca),
    ('adaboost_classifier', adaboost_classifier)
])
pipeline.fit(x_train_pca, y_train)
y_pred = pipeline.predict(x_test_pca)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

In [ ]:
#KNN 

from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score
base_classifier = KNeighborsClassifier(n_neighbors=5)
bagging_classifier = BaggingClassifier(base_estimator=base_classifier, n_estimators=50, random_state=42)
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('pca', pca),
    ('bagging_classifier', bagging_classifier)
])
pipeline.fit(x_train_pca, y_train)
y_pred = pipeline.predict(x_test_pca)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

In [ ]:
#GRADIENT BOOSTING

from sklearn.decomposition import PCA
from sklearn.metrics import r2_score
from sklearn.ensemble import GradientBoostingRegressor
GBR = GradientBoostingRegressor(n_estimators=500, learning_rate=0.07, max_depth=8, min_samples_split=4, min_samples_leaf=1, subsample=0.7, random_state=42)
GBR.fit(x_train_pca, y_train)
y_pred_test_original = GBR.predict(x_test_pca)
r2_test_original = r2_score(y_test, y_pred_test_original)
print(" R-squared score :", r2_test_original)


In [ ]:
#XG BOOST 

from sklearn.decomposition import PCA
from sklearn.metrics import r2_score
import xgboost as xgb
XGB = xgb.XGBRegressor(colsample_bytree=0.7, learning_rate=0.07, max_depth=8, min_child_weight=4, n_estimators=500, nthread=4, silent=True, subsample=0.7)
XGB.fit(x_train_pca, y_train)
y_pred_test_original = XGB.predict(x_test_pca)
r2_test_original = r2_score(y_test, y_pred_test_original)
print(" R-squared score :", r2_test_original)

In [ ]:
#TAKING LDA WITH ONE COMPONENT

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

# Assuming you have your data x_train, x_test

# Convert columns to string type if needed
x_train.columns = x_train.columns.astype(str)
x_test.columns = x_test.columns.astype(str)

# Define the LDA model with n_components=1
lda = LinearDiscriminantAnalysis(n_components=1)

# Fit LDA on the training data and transform both training and test data
x_train_lda = lda.fit_transform(x_train, y_train)
x_test_lda = lda.transform(x_test)


In [ ]:
#XG BOOST

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.metrics import r2_score
import xgboost as xgb


XGB = xgb.XGBRegressor(colsample_bytree=0.7, learning_rate=0.07, max_depth=8, min_child_weight=4, n_estimators=500, nthread=4, silent=True, subsample=0.7)
XGB.fit(x_train_lda, y_train)

# Make predictions on the test set
y_pred_test_original = XGB.predict(x_test_lda)

# Calculate R-squared score
r2_test_original = r2_score(y_test, y_pred_test_original)
print("R-squared score:", r2_test_original)



In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

# Define thresholds to convert continuous predictions to discrete classes
threshold_low = 0
threshold_high = 1  # You may adjust the thresholds based on your specific problem

# Convert continuous predictions to discrete classes based on the thresholds
y_pred_test_class = ((y_pred_test_original >= threshold_high) * 1).astype(int)

# Calculate precision, recall, and F1 score
precision = precision_score(y_test, y_pred_test_class, average='micro')  # Assuming binary classification
recall = recall_score(y_test, y_pred_test_class, average='micro')
f1 = f1_score(y_test, y_pred_test_class, average='micro')

print("Precision Score:", precision)
print("Recall Score:", recall)
print("F1 Score:", f1)


In [ ]:
#GRADIENT BOOSTING

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.metrics import r2_score
from sklearn.ensemble import GradientBoostingRegressor

# Define the Gradient Boosting Regressor
GBR = GradientBoostingRegressor(n_estimators=500, learning_rate=0.07, max_depth=8, min_samples_split=4, min_samples_leaf=1, subsample=0.7, random_state=42)

# Fit the model with LDA-transformed training data
GBR.fit(x_train_lda, y_train)

# Make predictions on the test set
y_pred_test_original = GBR.predict(x_test_lda)

# Calculate R-squared score
r2_test_original = r2_score(y_test, y_pred_test_original)
print("R-squared score:", r2_test_original)


In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

# Make predictions on the test set using the Gradient Boosting Regressor (GBR) model
y_pred_test_class = GBR.predict(x_test_lda)  # Assuming GBR is the trained Gradient Boosting Regressor model

# Define thresholds to convert continuous predictions to discrete classes
threshold_low = 0
threshold_high = 1  # You may adjust the thresholds based on your specific problem

# Convert continuous predictions to discrete classes based on the thresholds
y_pred_test_class = ((y_pred_test_class >= threshold_high) * 1).astype(int)

# Calculate precision, recall, and F1 score
precision = precision_score(y_test, y_pred_test_class, average='micro')  # Assuming binary classification
recall = recall_score(y_test, y_pred_test_class, average='micro')
f1 = f1_score(y_test, y_pred_test_class, average='micro')

print("Precision Score:", precision)
print("Recall Score:", recall)
print("F1 Score:", f1)


In [ ]:
#ADA BOOST

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score

# Assuming x_train and x_test are your training and testing data respectively
# Assuming y_train and y_test are your training and testing labels respectively

# Initialize the LDA transformer
lda = LDA()

# Define the base classifier
base_classifier = DecisionTreeClassifier(max_depth=3)

# Define the AdaBoost Classifier
adaboost_classifier = AdaBoostClassifier(base_estimator=base_classifier, n_estimators=50, random_state=42)

# Pipeline for preprocessing with LDA and AdaBoost Classifier
pipeline = Pipeline([
    ('scaler', StandardScaler()),  # Standardize features
    ('lda', lda),                  # Apply LDA for dimensionality reduction
    ('adaboost_classifier', adaboost_classifier)  # AdaBoost Classifier
])

# Train the model
pipeline.fit(x_train, y_train)

# Predict on the test set
y_pred = pipeline.predict(x_test)

# Calculate accuracy score
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

precision = precision_score(y_test, y_pred, average='macro')
recall = recall_score(y_test, y_pred, average='macro')
f1 = f1_score(y_test, y_pred, average='macro')

print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

In [ ]:
#BAGGING 

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split

# Assuming x_train and x_test are your training and testing data respectively
# Assuming y_train and y_test are your training and testing labels respectively

# Define the base classifier (Decision Tree)
base_classifier = DecisionTreeClassifier(max_depth=3)  # You can adjust the maximum depth as needed

# Define the Bagging Classifier
bagging_classifier = BaggingClassifier(base_estimator=base_classifier, n_estimators=50, random_state=42)

# Pipeline for preprocessing with Bagging Classifier
pipeline = Pipeline([
    ('scaler', StandardScaler()),            # Standardize features
    ('bagging_classifier', bagging_classifier)  # Bagging Classifier
])

# Train-test split

# Train the model
pipeline.fit(x_train, y_train)

# Predict on the test set
y_pred = pipeline.predict(x_test)

# Calculate accuracy score
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Calculate precision, recall, and F1 score
precision = precision_score(y_test, y_pred, average='macro')
recall = recall_score(y_test, y_pred, average='macro')
f1 = f1_score(y_test, y_pred, average='macro')

print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)


In [ ]:
#KNN

# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.ensemble import BaggingClassifier
# from sklearn.pipeline import Pipeline
# from sklearn.preprocessing import StandardScaler
# from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
# from sklearn.model_selection import train_test_split

# # Assuming x_train and x_test are your training and testing data respectively
# # Assuming y_train and y_test are your training and testing labels respectively

# # Define the base classifier (KNN)
# base_classifier = KNeighborsClassifier(n_neighbors=5)  # You can adjust the number of neighbors as needed

# # Define the Bagging Classifier
# bagging_classifier = BaggingClassifier(base_estimator=base_classifier, n_estimators=50, random_state=42)

# # Pipeline for preprocessing with Bagging Classifier
# pipeline = Pipeline([
#     ('scaler', StandardScaler()),            # Standardize features
#     ('bagging_classifier', bagging_classifier)  # Bagging Classifier
# ])

# # Train-test split


# # Train the model
# pipeline.fit(x_train, y_train)

# # Predict on the test set
# y_pred = pipeline.predict(x_test)

# # Calculate accuracy score
# accuracy = accuracy_score(y_test, y_pred)
# print("Accuracy:", accuracy)

# # Calculate precision, recall, and F1 score
# # precision = precision_score(y_test, y_pred, average='macro')
# # recall = recall_score(y_test, y_pred, average='macro')
# # f1 = f1_score(y_test, y_pred, average='macro')

# # print("Precision:", precision)
# # print("Recall:", recall)
# # print("F1 Score:", f1)


In [ ]:
#LDA WITH 2 COMPONENTS

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

# Assuming you have your data x_train, x_test

# Convert columns to string type if needed
x_train.columns = x_train.columns.astype(str)
x_test.columns = x_test.columns.astype(str)

# Define the LDA model with n_components=1
lda = LinearDiscriminantAnalysis(n_components=2)

# Fit LDA on the training data and transform both training and test data
x_train_lda = lda.fit_transform(x_train, y_train)
x_test_lda = lda.transform(x_test)


In [ ]:
#XG BOOST

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.metrics import r2_score
import xgboost as xgb


XGB = xgb.XGBRegressor(colsample_bytree=0.7, learning_rate=0.07, max_depth=8, min_child_weight=4, n_estimators=500, nthread=4, silent=True, subsample=0.7)
XGB.fit(x_train_lda, y_train)

# Make predictions on the test set
y_pred_test_original = XGB.predict(x_test_lda)

# Calculate R-squared score
r2_test_original = r2_score(y_test, y_pred_test_original)
print("R-squared score:", r2_test_original)


In [ ]:
#GRADIENT BOOSTING

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.metrics import r2_score
from sklearn.ensemble import GradientBoostingRegressor

# Define the Gradient Boosting Regressor
GBR = GradientBoostingRegressor(n_estimators=500, learning_rate=0.07, max_depth=8, min_samples_split=4, min_samples_leaf=1, subsample=0.7, random_state=42)

# Fit the model with LDA-transformed training data
GBR.fit(x_train_lda, y_train)

# Make predictions on the test set
y_pred_test_original = GBR.predict(x_test_lda)
A
# Calculate R-squared score
r2_test_original = r2_score(y_test, y_pred_test_original)
print("R-squared score:", r2_test_original)


In [ ]:
#ADA BOOST

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score

# Assuming x_train and x_test are your training and testing data respectively
# Assuming y_train and y_test are your training and testing labels respectively

# Initialize the LDA transformer
lda = LDA()

# Define the base classifier
base_classifier = DecisionTreeClassifier(max_depth=3)

# Define the AdaBoost Classifier
adaboost_classifier = AdaBoostClassifier(base_estimator=base_classifier, n_estimators=50, random_state=42)

# Pipeline for preprocessing with LDA and AdaBoost Classifier
pipeline = Pipeline([
    ('scaler', StandardScaler()),  # Standardize features
    ('lda', lda),                  # Apply LDA for dimensionality reduction
    ('adaboost_classifier', adaboost_classifier)  # AdaBoost Classifier
])

# Train the model
pipeline.fit(x_train, y_train)

# Predict on the test set
y_pred = pipeline.predict(x_test)

# Calculate accuracy score
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# precision = precision_score(y_test, y_pred, average='macro')
# recall = recall_score(y_test, y_pred, average='macro')
# f1 = f1_score(y_test, y_pred, average='macro')

# print("Precision:", precision)
# print("Recall:", recall)
# print("F1 Score:", f1)

In [ ]:
#KNN

from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split

# Assuming x_train and x_test are your training and testing data respectively
# Assuming y_train and y_test are your training and testing labels respectively

# Define the base classifier (KNN)
base_classifier = KNeighborsClassifier(n_neighbors=5)  # You can adjust the number of neighbors as needed

# Define the Bagging Classifier
bagging_classifier = BaggingClassifier(base_estimator=base_classifier, n_estimators=50, random_state=42)

# Pipeline for preprocessing with Bagging Classifier
pipeline = Pipeline([
    ('scaler', StandardScaler()),            # Standardize features
    ('bagging_classifier', bagging_classifier)  # Bagging Classifier
])

# Train-test split


# Train the model
pipeline.fit(x_train, y_train)

# Predict on the test set
y_pred = pipeline.predict(x_test)

# Calculate accuracy score
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Calculate precision, recall, and F1 score
precision = precision_score(y_test, y_pred, average='macro')
recall = recall_score(y_test, y_pred, average='macro')
f1 = f1_score(y_test, y_pred, average='macro')

print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)


In [ ]:
#LDA WITH 3 COMPONENTS

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

# Assuming you have your data x_train, x_test

# Convert columns to string type if needed
x_train.columns = x_train.columns.astype(str)
x_test.columns = x_test.columns.astype(str)

# Define the LDA model with n_components=1
lda = LinearDiscriminantAnalysis(n_components=3)

# Fit LDA on the training data and transform both training and test data
x_train_lda = lda.fit_transform(x_train, y_train)
x_test_lda = lda.transform(x_test)


In [ ]:
#XG BOOST

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.metrics import r2_score
import xgboost as xgb


XGB = xgb.XGBRegressor(colsample_bytree=0.7, learning_rate=0.07, max_depth=8, min_child_weight=4, n_estimators=500, nthread=4, silent=True, subsample=0.7)
XGB.fit(x_train_lda, y_train)

# Make predictions on the test set
y_pred_test_original = XGB.predict(x_test_lda)

# Calculate R-squared score
r2_test_original = r2_score(y_test, y_pred_test_original)
print("R-squared score:", r2_test_original)


In [ ]:
#GRADIENT BOOSTING

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.metrics import r2_score
from sklearn.ensemble import GradientBoostingRegressor

# Define the Gradient Boosting Regressor
GBR = GradientBoostingRegressor(n_estimators=500, learning_rate=0.07, max_depth=8, min_samples_split=4, min_samples_leaf=1, subsample=0.7, random_state=42)

# Fit the model with LDA-transformed training data
GBR.fit(x_train_lda, y_train)

# Make predictions on the test set
y_pred_test_original = GBR.predict(x_test_lda)

# Calculate R-squared score
r2_test_original = r2_score(y_test, y_pred_test_original)
print("R-squared score:", r2_test_original)


In [ ]:
#ADA BOOST

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score

# Assuming x_train and x_test are your training and testing data respectively
# Assuming y_train and y_test are your training and testing labels respectively

# Initialize the LDA transformer
lda = LDA()

# Define the base classifier
base_classifier = DecisionTreeClassifier(max_depth=3)

# Define the AdaBoost Classifier
adaboost_classifier = AdaBoostClassifier(base_estimator=base_classifier, n_estimators=50, random_state=42)

# Pipeline for preprocessing with LDA and AdaBoost Classifier
pipeline = Pipeline([
    ('scaler', StandardScaler()),  # Standardize features
    ('lda', lda),                  # Apply LDA for dimensionality reduction
    ('adaboost_classifier', adaboost_classifier)  # AdaBoost Classifier
])

# Train the model
pipeline.fit(x_train, y_train)

# Predict on the test set
y_pred = pipeline.predict(x_test)

# Calculate accuracy score
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# precision = precision_score(y_test, y_pred, average='macro')
# recall = recall_score(y_test, y_pred, average='macro')
# f1 = f1_score(y_test, y_pred, average='macro')

# print("Precision:", precision)
# print("Recall:", recall)
# print("F1 Score:", f1)

In [ ]:
#KNN 

from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split

# Assuming x_train and x_test are your training and testing data respectively
# Assuming y_train and y_test are your training and testing labels respectively

# Define the base classifier (KNN)
base_classifier = KNeighborsClassifier(n_neighbors=5)  # You can adjust the number of neighbors as needed

# Define the Bagging Classifier
bagging_classifier = BaggingClassifier(base_estimator=base_classifier, n_estimators=50, random_state=42)

# Pipeline for preprocessing with Bagging Classifier
pipeline = Pipeline([
    ('scaler', StandardScaler()),            # Standardize features
    ('bagging_classifier', bagging_classifier)  # Bagging Classifier
])

# Train-test split


# Train the model
pipeline.fit(x_train, y_train)

# Predict on the test set
y_pred = pipeline.predict(x_test)

# Calculate accuracy score
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Calculate precision, recall, and F1 score
precision = precision_score(y_test, y_pred, average='macro')
recall = recall_score(y_test, y_pred, average='macro')
f1 = f1_score(y_test, y_pred, average='macro')

print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)


In [ ]:
#BAGGING

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split

# Assuming x_train and x_test are your training and testing data respectively
# Assuming y_train and y_test are your training and testing labels respectively

# Define the base classifier (Decision Tree)
base_classifier = DecisionTreeClassifier(max_depth=3)  # You can adjust the maximum depth as needed

# Define the Bagging Classifier
bagging_classifier = BaggingClassifier(base_estimator=base_classifier, n_estimators=50, random_state=42)

# Pipeline for preprocessing with Bagging Classifier
pipeline = Pipeline([
    ('scaler', StandardScaler()),            # Standardize features
    ('bagging_classifier', bagging_classifier)  # Bagging Classifier
])

# Train-test split

# Train the model
pipeline.fit(x_train, y_train)

# Predict on the test set
y_pred = pipeline.predict(x_test)

# Calculate accuracy score
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Calculate precision, recall, and F1 score
# precision = precision_score(y_test, y_pred, average='macro')
# recall = recall_score(y_test, y_pred, average='macro')
# f1 = f1_score(y_test, y_pred, average='macro')

# print("Precision:", precision)
# print("Recall:", recall)
# print("F1 Score:", f1)


In [ ]:
#LDA WITH FOUR COMPONENTS

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

# Assuming you have your data x_train, x_test

# Convert columns to string type if needed
x_train.columns = x_train.columns.astype(str)
x_test.columns = x_test.columns.astype(str)

# Define the LDA model with n_components=1
lda = LinearDiscriminantAnalysis(n_components=4)

# Fit LDA on the training data and transform both training and test data
x_train_lda = lda.fit_transform(x_train, y_train)
x_test_lda = lda.transform(x_test)


In [ ]:
#XG BOOST

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.metrics import r2_score
import xgboost as xgb


XGB = xgb.XGBRegressor(colsample_bytree=0.7, learning_rate=0.07, max_depth=8, min_child_weight=4, n_estimators=500, nthread=4, silent=True, subsample=0.7)
XGB.fit(x_train_lda, y_train)

# Make predictions on the test set
y_pred_test_original = XGB.predict(x_test_lda)

# Calculate R-squared score
r2_test_original = r2_score(y_test, y_pred_test_original)
print("R-squared score:", r2_test_original)


In [ ]:
#GRADIENT BOOSTING 

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.metrics import r2_score
from sklearn.ensemble import GradientBoostingRegressor

# Define the Gradient Boosting Regressor
GBR = GradientBoostingRegressor(n_estimators=500, learning_rate=0.07, max_depth=8, min_samples_split=4, min_samples_leaf=1, subsample=0.7, random_state=42)

# Fit the model with LDA-transformed training data
GBR.fit(x_train_lda, y_train)

# Make predictions on the test set
y_pred_test_original = GBR.predict(x_test_lda)

# Calculate R-squared score
r2_test_original = r2_score(y_test, y_pred_test_original)
print("R-squared score:", r2_test_original)


In [ ]:
#ADA BOOST

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score

# Assuming x_train and x_test are your training and testing data respectively
# Assuming y_train and y_test are your training and testing labels respectively

# Initialize the LDA transformer
lda = LDA()

# Define the base classifier
base_classifier = DecisionTreeClassifier(max_depth=3)

# Define the AdaBoost Classifier
adaboost_classifier = AdaBoostClassifier(base_estimator=base_classifier, n_estimators=50, random_state=42)

# Pipeline for preprocessing with LDA and AdaBoost Classifier
pipeline = Pipeline([
    ('scaler', StandardScaler()),  # Standardize features
    ('lda', lda),                  # Apply LDA for dimensionality reduction
    ('adaboost_classifier', adaboost_classifier)  # AdaBoost Classifier
])

# Train the model
pipeline.fit(x_train, y_train)

# Predict on the test set
y_pred = pipeline.predict(x_test)

# Calculate accuracy score
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# precision = precision_score(y_test, y_pred, average='macro')
# recall = recall_score(y_test, y_pred, average='macro')
# f1 = f1_score(y_test, y_pred, average='macro')

# print("Precision:", precision)
# print("Recall:", recall)
# print("F1 Score:", f1)

In [ ]:
#KNN

from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split

# Assuming x_train and x_test are your training and testing data respectively
# Assuming y_train and y_test are your training and testing labels respectively

# Define the base classAifier (KNN)
base_classifier = KNeighborsClassifier(n_neighbors=5)  # You can adjust the number of neighbors as needed

# Define the Bagging Classifier
bagging_classifier = BaggingClassifier(base_estimator=base_classifier, n_estimators=50, random_state=42)

# Pipeline for preprocessing with Bagging Classifier
pipeline = Pipeline([
    ('scaler', StandardScaler()),            # Standardize features
    ('bagging_classifier', bagging_classifier)  # Bagging Classifier
])

# Train-test split


# Train the model
pipeline.fit(x_train, y_train)

# Predict on the test set
y_pred = pipeline.predict(x_test)

# Calculate accuracy score
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Calculate precision, recall, and F1 score
precision = precision_score(y_test, y_pred, average='macro')
recall = recall_score(y_test, y_pred, average='macro')
f1 = f1_score(y_test, y_pred, average='macro')

print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
A

In [ ]:
#BAGGING

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split

# Assuming x_train and x_test are your training and testing data respectively
# Assuming y_train and y_test are your training and testing labels respectively

# Define the base classifier (Decision Tree)
base_classifier = DecisionTreeClassifier(max_depth=3)  # You can adjust the maximum depth as needed

# Define the Bagging Classifier
bagging_classifier = BaggingClassifier(base_estimator=base_classifier, n_estimators=50, random_state=42)

# Pipeline for preprocessing with Bagging Classifier
pipeline = Pipeline([
    ('scaler', StandardScaler()),            # Standardize features
    ('bagging_classifier', bagging_classifier)  # Bagging Classifier
])

# Train-test split

# Train the model
pipeline.fit(x_train, y_train)

# Predict on the test set
y_pred = pipeline.predict(x_test)

# Calculate accuracy score
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Calculate precision, recall, and F1 score
# precision = precision_score(y_test, y_pred, average='macro')
# recall = recall_score(y_test, y_pred, average='macro')
# f1 = f1_score(y_test, y_pred, average='macro')

# print("Precision:", precision)
# print("Recall:", recall)
# print("F1 Score:", f1)


In [ ]:
#LDA WITH 5 COMPONENTS

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

# Assuming you have your data x_train, x_test

# Convert columns to string type if needed
x_train.columns = x_train.columns.astype(str)
x_test.columns = x_test.columns.astype(str)

# Define the LDA model with n_components=1
lda = LinearDiscriminantAnalysis(n_components=5)

# Fit LDA on the training data and transform both training and test data
x_train_lda = lda.fit_transform(x_train, y_train)
x_test_lda = lda.transform(x_test)


In [ ]:
#XG BOOST

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.metrics import r2_score
import xgboost as xgb


XGB = xgb.XGBRegressor(colsample_bytree=0.7, learning_rate=0.07, max_depth=8, min_child_weight=4, n_estimators=500, nthread=4, silent=True, subsample=0.7)
XGB.fit(x_train_lda, y_train)

# Make predictions on the test set
y_pred_test_original = XGB.predict(x_test_lda)

# Calculate R-squared score
r2_test_original = r2_score(y_test, y_pred_test_original)
print("R-squared score:", r2_test_original)



In [ ]:
#GRADIENT BOOSTING

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.metrics import r2_score
from sklearn.ensemble import GradientBoostingRegressor

# Define the Gradient Boosting Regressor
GBR = GradientBoostingRegressor(n_estimators=500, learning_rate=0.07, max_depth=8, min_samples_split=4, min_samples_leaf=1, subsample=0.7, random_state=42)

# Fit the model with LDA-transformed training data
GBR.fit(x_train_lda, y_train)

# Make predictions on the test set
y_pred_test_original = GBR.predict(x_test_lda)

# Calculate R-squared score
r2_test_original = r2_score(y_test, y_pred_test_original)
print("R-squared score:", r2_test_original)


In [ ]:
#ADA BOOST

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score

# Assuming x_train and x_test are your training and testing data respectively
# Assuming y_train and y_test are your training and testing labels respectively

# Initialize the LDA transformer
lda = LDA()

# Define the base classifier
base_classifier = DecisionTreeClassifier(max_depth=3)

# Define the AdaBoost Classifier
adaboost_classifier = AdaBoostClassifier(base_estimator=base_classifier, n_estimators=50, random_state=42)

# Pipeline for preprocessing with LDA and AdaBoost Classifier
pipeline = Pipeline([
    ('scaler', StandardScaler()),  # Standardize features
    ('lda', lda),                  # Apply LDA for dimensionality reduction
    ('adaboost_classifier', adaboost_classifier)  # AdaBoost Classifier
])

# Train the model
pipeline.fit(x_train, y_train)

# Predict on the test set
y_pred = pipeline.predict(x_test)

# Calculate accuracy score
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# precision = precision_score(y_test, y_pred, average='macro')
# recall = recall_score(y_test, y_pred, average='macro')
# f1 = f1_score(y_test, y_pred, average='macro')

# print("Precision:", precision)
# print("Recall:", recall)
# print("F1 Score:", f1)

In [ ]:
#KNN 

from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split

# Assuming x_train and x_test are your training and testing data respectively
# Assuming y_train and y_test are your training and testing labels respectively

# Define the base classifier (KNN)
base_classifier = KNeighborsClassifier(n_neighbors=5)  # You can adjust the number of neighbors as needed

# Define the Bagging Classifier
bagging_classifier = BaggingClassifier(base_estimator=base_classifier, n_estimators=50, random_state=42)

# Pipeline for preprocessing with Bagging Classifier
pipeline = Pipeline([
    ('scaler', StandardScaler()),            # Standardize features
    ('bagging_classifier', bagging_classifier)  # Bagging Classifier
])

# Train-test split


# Train the model
pipeline.fit(x_train, y_train)

# Predict on the test set
y_pred = pipeline.predict(x_test)

# Calculate accuracy score
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Calculate precision, recall, and F1 score
precision = precision_score(y_test, y_pred, average='macro')
recall = recall_score(y_test, y_pred, average='macro')
f1 = f1_score(y_test, y_pred, average='macro')

print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)


In [ ]:
#BAGGING 

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split

# Assuming x_train and x_test are your training and testing data respectively
# Assuming y_train and y_test are your training and testing labels respectively

# Define the base classifier (Decision Tree)
base_classifier = DecisionTreeClassifier(max_depth=3)  # You can adjust the maximum depth as needed

# Define the Bagging Classifier
bagging_classifier = BaggingClassifier(base_estimator=base_classifier, n_estimators=50, random_state=42)

# Pipeline for preprocessing with Bagging Classifier
pipeline = Pipeline([
    ('scaler', StandardScaler()),            # Standardize features
    ('bagging_classifier', bagging_classifier)  # Bagging Classifier
])

# Train-test split

# Train the model
pipeline.fit(x_train, y_train)

# Predict on the test set
y_pred = pipeline.predict(x_test)

# Calculate accuracy score
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Calculate precision, recall, and F1 score
# precision = precision_score(y_test, y_pred, average='macro')
# recall = recall_score(y_test, y_pred, average='macro')
# f1 = f1_score(y_test, y_pred, average='macro')

# print("Precision:", precision)
# print("Recall:", recall)
# print("F1 Score:", f1)


In [ ]:
#UN VARIENT

from sklearn.feature_selection import SelectKBest, f_classif

# Assuming you have your data x_train, x_test

# Initialize SelectKBest with the desired number of features (k)
k = 5 # You can adjust the number of features as needed
selector = SelectKBest(score_func=f_classif, k=k)

# Fit the selector on the training data and transform both training and test data
x_train_selected = selector.fit_transform(x_train, y_train)
x_test_selected = selector.transform(x_test)


In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

# Assuming x_train and x_test are your training and testing data respectively
# Assuming y_train and y_test are your training and testing labels respectively

# Define the base classifier (Decision Tree)
base_classifier = DecisionTreeClassifier(max_depth=3)  # You can adjust the maximum depth as needed

# Define the Bagging Classifier
bagging_classifier = BaggingClassifier(base_estimator=base_classifier, n_estimators=50, random_state=42)

# Define the feature selector (SelectKBest with f_classif score function)
selector = SelectKBest(score_func=f_classif, k=9)  # Adjust the number of features as needed

# Pipeline for preprocessing with Bagging Classifier and Univariate Feature Selection
pipeline = Pipeline([
    ('scaler', StandardScaler()),            # Standardize features
    ('selector', selector),                  # Feature selection
    ('bagging_classifier', bagging_classifier)  # Bagging Classifier
])

# Train-test split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

# Train the model
pipeline.fit(x_train, y_train)

# Predict on the test set
y_pred = pipeline.predict(x_test)

# Calculate accuracy score
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split

# Assuming x_train and x_test are your training and testing data respectively
# Assuming y_train and y_test are your training and testing labels respectively

# Define the base classifier (KNN)
base_classifier = KNeighborsClassifier(n_neighbors=5)  # You can adjust the number of neighbors as needed

# Define the feature selector (SelectKBest with f_classif score function)
selector = SelectKBest(score_func=f_classif, k=9)  # Adjust the number of features as needed

# Pipeline for preprocessing with KNN Classifier and Univariate Feature Selection
pipeline = Pipeline([
    ('selector', selector),                  # Feature selection
    ('classifier', base_classifier)          # KNN Classifier
])

# Train-test split
x_train, x_test, y_train, y_test = train_test_split(x_test, y_test, test_size=0.3, random_state=42)

# Train the model
pipeline.fit(x_train, y_train)

# Predict on the test set
y_pred = pipeline.predict(x_test)

# Calculate accuracy score
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Calculate precision, recall, and F1 score
precision = precision_score(y_test, y_pred, average='macro')
recall = recall_score(y_test, y_pred, average='macro')
f1 = f1_score(y_test, y_pred, average='macro')

print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)


In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split

# Assuming x_train and x_test are your training and testing data respectively
# Assuming y_train and y_test are your training and testing labels respectively

# Define the base classifier (KNN)
base_classifier = AdaBoostClassifier(n_estimators=50, random_state=42)  # You can adjust the number of estimators as needed

# Define the feature selector (SelectKBest with f_classif score function)
selector = SelectKBest(score_func=f_classif, k=7)  # Adjust the number of features as needed

# Pipeline for preprocessing with AdaBoost Classifier and Univariate Feature Selection
pipeline = Pipeline([
    ('selector', selector),                  # Feature selection
    ('classifier', base_classifier)          # AdaBoost Classifier
])

# Train-test split
x_train, x_test, y_train, y_test = train_test_split(x_test, y_test, test_size=0.3, random_state=42)

# Train the model
pipeline.fit(x_train, y_train)

# Predict on the test set
y_pred = pipeline.predict(x_test)

# Calculate accuracy score
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Calculate precision, recall, and F1 score
precision = precision_score(y_test, y_pred, average='macro')
recall = recall_score(y_test, y_pred, average='macro')
f1 = f1_score(y_test, y_pred, average='macro')

print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)


In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split

# Assuming x_train and x_test are your training and testing data respectively
# Assuming y_train and y_test are your training and testing labels respectively

# Define the base classifier (Gradient Boosting)
base_classifier = GradientBoostingClassifier(n_estimators=50, random_state=42)  # You can adjust the number of estimators as needed

# Define the feature selector (SelectKBest with f_classif score function)
selector = SelectKBest(score_func=f_classif, k=11)  # Adjust the number of features as needed

# Pipeline for preprocessing with Gradient Boosting Classifier and Univariate Feature Selection
pipeline = Pipeline([
    ('selector', selector),                  # Feature selection
    ('classifier', base_classifier)          # Gradient Boosting Classifier
])

# Train-test split
x_train, x_test, y_train, y_test = train_test_split(x_test, y_test, test_size=0.3, random_state=42)
# Convert column names to string type if they are not already
x_train.columns = x_train.columns.astype(str)
x_test.columns = x_test.columns.astype(str)

# Train the model
pipeline.fit(x_train, y_train)

# Predict on the test set
y_pred = pipeline.predict(x_test)

# Calculate accuracy score
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Calculate precision, recall, and F1 score
precision = precision_score(y_test, y_pred, average='macro')
recall = recall_score(y_test, y_pred, average='macro')
f1 = f1_score(y_test, y_pred, average='macro')

print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
